<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/BDSN_DailyShow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

Apache Spark: An Introduction
https://www.dataquest.io/blog/apache-spark/

#Get Data

In [1]:
#import urllib.request
#urllib.request.urlretrieve('https://raw.githubusercontent.com/fivethirtyeight/data/master/daily-show-guests/daily_show_guests.csv','dsg.csv')
#!wget 'https://raw.githubusercontent.com/fivethirtyeight/data/master/daily-show-guests/daily_show_guests.csv'
#!gdown https://drive.google.com/uc?id=1UQWzVXAg2ytuAmtizEsWx6E1D38XUBX4
!wget -q  'https://raw.githubusercontent.com/Praxis-QR/BDSN/main/data/dailyshow/daily_show_guests.csv'

# Setup

## This approach is not required anymore -- deprecated

In [ ]:
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://apache.osuosl.org/spark/spark-2.2.2/spark-2.2.2-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#
#!tar xf spark-2.4.0-bin-hadoop2.7.tgz
#!tar xf spark-2.4.3-bin-hadoop2.7.tgz
#!tar xf spark-2.4.4-bin-hadoop2.7.tgz
#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark
!pip install -q pyspark

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 49.8 MB/s 


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

## Direct PIP Approach 

In [2]:
!pip3 -q install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
#import findspark
#findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
#from pyspark.sql import SparkSession
#spark = SparkSession.builder.master("local[*]").getOrCreate()
# note UI port switched from default 4040 to 4050 to avoid clash with ngrok
#spark = SparkSession.builder.master("local[*]").config('spark.ui.port', '4050').getOrCreate()

In [4]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [5]:
raw_data = sc.textFile("daily_show_guests.csv")
raw_data.take(5)

['YEAR,GoogleKnowlege_Occupation,Show,Group,Raw_Guest_List',
 '1999,actor,1/11/99,Acting,Michael J. Fox',
 '1999,Comedian,1/12/99,Comedy,Sandra Bernhard',
 '1999,television actress,1/13/99,Acting,Tracey Ullman',
 '1999,film actress,1/14/99,Acting,Gillian Anderson']

# Daily Show

In [6]:
#
# non Pythonic, non Spark of doing things
#
import csv

f = open('daily_show_guests.csv')
tally_D = dict()
for line in f:
    data_line = line.split(',')
    year = data_line[0]
    if year in tally_D.keys():
       tally_D[year] = tally_D[year] + 1
    else:
       tally_D[year] = 1

print(tally_D)

{'YEAR': 1, '1999': 166, '2000': 169, '2001': 157, '2002': 159, '2003': 166, '2004': 164, '2005': 162, '2006': 161, '2007': 141, '2008': 164, '2009': 163, '2010': 165, '2011': 163, '2012': 164, '2013': 166, '2014': 163, '2015': 100}


## Introducing Spark

In [7]:
daily_show = raw_data.map(lambda line: line.split(','))
daily_show.take(5)

[['YEAR', 'GoogleKnowlege_Occupation', 'Show', 'Group', 'Raw_Guest_List'],
 ['1999', 'actor', '1/11/99', 'Acting', 'Michael J. Fox'],
 ['1999', 'Comedian', '1/12/99', 'Comedy', 'Sandra Bernhard'],
 ['1999', 'television actress', '1/13/99', 'Acting', 'Tracey Ullman'],
 ['1999', 'film actress', '1/14/99', 'Acting', 'Gillian Anderson']]

In [8]:
tallyMap = daily_show.map(lambda x: (x[0], 1))
tallyMap.take(5)


[('YEAR', 1), ('1999', 1), ('1999', 1), ('1999', 1), ('1999', 1)]

In [9]:
tallyRed = tallyMap.reduceByKey(lambda x,y: x+y)
tallyRed.take(5)

[('YEAR', 1), ('2002', 159), ('2003', 166), ('2004', 164), ('2007', 141)]

In [10]:
tally = daily_show.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y)
print(tally)

PythonRDD[14] at RDD at PythonRDD.scala:53


In [11]:
tally.count()

18

In [12]:
tally.take(tally.count())

[('YEAR', 1),
 ('2002', 159),
 ('2003', 166),
 ('2004', 164),
 ('2007', 141),
 ('2010', 165),
 ('2011', 163),
 ('2012', 164),
 ('2013', 166),
 ('2014', 163),
 ('2015', 100),
 ('1999', 166),
 ('2000', 169),
 ('2001', 157),
 ('2005', 162),
 ('2006', 161),
 ('2008', 164),
 ('2009', 163)]

In [13]:
def filter_year(line):
    if line[0] == 'YEAR':
        return False
    else:
        return True
filtered_daily_show = daily_show.filter(lambda line: filter_year(line))

In [14]:
tally2 = filtered_daily_show.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y)
print(tally2)

PythonRDD[23] at RDD at PythonRDD.scala:53


In [15]:
tally2.take(tally2.count())

[('2002', 159),
 ('2003', 166),
 ('2004', 164),
 ('2007', 141),
 ('2010', 165),
 ('2011', 163),
 ('2012', 164),
 ('2013', 166),
 ('2014', 163),
 ('2015', 100),
 ('1999', 166),
 ('2000', 169),
 ('2001', 157),
 ('2005', 162),
 ('2006', 161),
 ('2008', 164),
 ('2009', 163)]

In [16]:
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .take(20)

[('actor', 596),
 ('film actress', 21),
 ('model', 9),
 ('stand-up comedian', 44),
 ('actress', 271),
 ('television personality', 13),
 ('comic', 2),
 ('musician', 19),
 ('film actor', 19),
 ('journalist', 253),
 ('us senator', 50),
 ('vocalist', 2),
 ('film director', 24),
 ('singer', 23),
 ('muppet', 2),
 ('american television personality', 3),
 ('rapper', 10),
 ('football player', 1),
 ('former mayor of cincinatti', 2),
 ('businesswoman', 2)]

## Extra Sorting

In [17]:
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .take(20)

[('actor', 596),
 ('film actress', 21),
 ('model', 9),
 ('stand-up comedian', 44),
 ('actress', 271),
 ('television personality', 13),
 ('comic', 2),
 ('musician', 19),
 ('film actor', 19),
 ('journalist', 253),
 ('us senator', 50),
 ('vocalist', 2),
 ('film director', 24),
 ('singer', 23),
 ('muppet', 2),
 ('american television personality', 3),
 ('rapper', 10),
 ('football player', 1),
 ('former mayor of cincinatti', 2),
 ('businesswoman', 2)]

In [18]:
#sorted by profession 
#not sorted by number of appearances
# in sortbykey, 1st parameter determines direction of sort, second parameter is optional)
# in sortbykey, 2nd parameter determines number of partition in output RDD
# see next two cells
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .sortByKey(0, 1) \
                   .take(20)

[('writer', 52),
 ('white house sommunications director', 5),
 ('white house official', 1),
 ('vocalist', 2),
 ('vice president', 2),
 ('us senator', 50),
 ('us secretary of energy', 1),
 ('us secretary of defense', 1),
 ('us secetary of education', 1),
 ('us representative', 9),
 ('us president', 10),
 ('us permanent representative to nato', 1),
 ('us official', 1),
 ('us assistant attorney', 1),
 ('university professor', 2),
 ('united states senator', 15),
 ('united states senate member', 4),
 ('united states secretary of the navy', 1),
 ('united states secretary of housing and urban development', 1),
 ('united states secretary of defense', 1)]

In [19]:
#sorted by profession 
#not sorted by number of appearances
# in sortbykey, 1st parameter determines direction of sort, second parameter is optional)
# in sortbykey, 2nd parameter determines number of partition in output RDD
# see next two cells
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .sortByKey(1, 1) \
                   .take(20)

[('-', 1),
 ('0', 4),
 ('academic', 6),
 ('accountant', 1),
 ('activist', 14),
 ('actor', 596),
 ('actress', 271),
 ('administrator', 1),
 ('admiral', 2),
 ('adviser', 4),
 ('advocate', 1),
 ('aei president', 1),
 ('afghan politician', 1),
 ('american football quarterback', 3),
 ('american football running back', 1),
 ('american football wide receiver', 2),
 ('american football wide reciever', 1),
 ('american political figure', 2),
 ('american politician', 8),
 ('american television personality', 3)]

In [20]:
!rm -r dumpDir
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .sortByKey(0,2)\
                   .saveAsTextFile("dumpDir")

rm: cannot remove 'dumpDir': No such file or directory


In [21]:
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .take(10)

[('actor', 596),
 ('film actress', 21),
 ('model', 9),
 ('stand-up comedian', 44),
 ('actress', 271),
 ('television personality', 13),
 ('comic', 2),
 ('musician', 19),
 ('film actor', 19),
 ('journalist', 253)]

In [22]:
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .map(lambda x: (x[1],x[0]))\
                   .take(10)

[(596, 'actor'),
 (21, 'film actress'),
 (9, 'model'),
 (44, 'stand-up comedian'),
 (271, 'actress'),
 (13, 'television personality'),
 (2, 'comic'),
 (19, 'musician'),
 (19, 'film actor'),
 (253, 'journalist')]

In [23]:
#to sort by number of appearances, we first need to flip the key and value 
#

# sort 1st arg configures ascending sort, 2nd configures 1 task
# https://stackoverflow.com/questions/38569052/how-do-i-invert-key-and-value-in-rdd-in-python-3-pyspark

filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .map(lambda x: (x[1],x[0]))\
                   .sortByKey(0, 1) \
                   .map(lambda x: (x[1],x[0])) \
                   .take(10)

[('actor', 596),
 ('actress', 271),
 ('journalist', 253),
 ('author', 152),
 ('comedian', 103),
 ('writer', 52),
 ('us senator', 50),
 ('stand-up comedian', 44),
 ('television host', 39),
 ('professor', 37)]

In [24]:
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .sortBy(lambda x : -1*x[1])\
                   .take(10)

[('actor', 596),
 ('actress', 271),
 ('journalist', 253),
 ('author', 152),
 ('comedian', 103),
 ('writer', 52),
 ('us senator', 50),
 ('stand-up comedian', 44),
 ('television host', 39),
 ('professor', 37)]

In [25]:
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .takeOrdered(10, lambda aTuple:-1*aTuple[1])

[('actor', 596),
 ('actress', 271),
 ('journalist', 253),
 ('author', 152),
 ('comedian', 103),
 ('writer', 52),
 ('us senator', 50),
 ('stand-up comedian', 44),
 ('television host', 39),
 ('professor', 37)]

In [26]:
from datetime import datetime
import pytz
print('signed off at  ',datetime.now(pytz.timezone('Asia/Kolkata')))

signed off at   2023-05-08 05:58:36.032639+05:30


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)